In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os



# Import API key
from api_keys import g_key

merge CSV files into a combined dataframe for analysis

In [3]:
file = "output/my_combined.csv"

county_df = pd.read_csv(file)
county_df.head()

file2 = "Resources/countycoordinates.xlsx"

coord_df = pd.read_excel(file2)
clean_coord = coord_df[["State", "County", "Latitude", "Longitude"]]
clean_coord

data_loc = pd.DataFrame.merge(county_df, clean_coord, on=["State", "County"])



#data_loc.to_csv('output/clean_with_lat_long.csv', index = False)

data_loc.head()

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,# Deaths,Years of Potential Life Lost Rate,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude
0,Alabama,Autauga,54571,20221,13.858333,61082.916667,2,2,1,2,...,830.0,9158.0,34,18,711.0,145.0,1327.0,19.0,32.534920,-86.642749
1,Alabama,Baldwin,182265,73180,14.358065,60664.935484,3,3,3,2,...,2573.0,7394.0,27,16,2615.0,415.0,4791.0,20.0,30.660970,-87.749840
2,Alabama,Barbour,27457,9820,24.533333,43123.000000,3,3,1,2,...,413.0,8573.0,44,25,525.0,85.0,771.0,45.0,31.869603,-85.393197
3,Alabama,Bibb,22915,7953,16.025000,43362.500000,0,0,0,1,...,407.0,11766.0,40,20,333.0,33.0,561.0,29.0,32.998644,-87.126439
4,Alabama,Blount,57322,21578,17.911111,52136.000000,0,0,0,2,...,906.0,9333.0,35,19,705.0,123.0,1316.0,22.0,33.980867,-86.567371


In [4]:
#add additional data column to dataframe and merge again with medicare data

file = "output/clean_with_lat_long.csv"

longlat_df = pd.read_csv(file)
longlat_df.head()

file2 = "Resources/Medicare_Enrollment_2017.csv"

health_df = pd.read_csv(file2)

clean_data2 = pd.DataFrame.merge(longlat_df, health_df, on=["State", "County"])



#clean_data2.to_csv('output/clean_data2.csv', index = False)

clean_data2.head()

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,Years of Potential Life Lost Rate,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude,Original Medicare
0,Alabama,Autauga,54571,20221,13.858333,61082.916667,2,2,1,2,...,9158.0,34,18,711.0,145.0,1327.0,19.0,32.534920,-86.642749,5784
1,Alabama,Baldwin,182265,73180,14.358065,60664.935484,3,3,3,2,...,7394.0,27,16,2615.0,415.0,4791.0,20.0,30.660970,-87.749840,28388
2,Alabama,Barbour,27457,9820,24.533333,43123.000000,3,3,1,2,...,8573.0,44,25,525.0,85.0,771.0,45.0,31.869603,-85.393197,4372
3,Alabama,Bibb,22915,7953,16.025000,43362.500000,0,0,0,1,...,11766.0,40,20,333.0,33.0,561.0,29.0,32.998644,-87.126439,2480
4,Alabama,Blount,57322,21578,17.911111,52136.000000,0,0,0,2,...,9333.0,35,19,705.0,123.0,1316.0,22.0,33.980867,-86.567371,6070


In [5]:
#print column to see data

clean_data2["Food Environment Index"]


0       6.9
1       7.5
2       4.9
3       7.3
4       8.3
       ... 
3088    8.1
3089    8.0
3090    7.5
3091    8.0
3092    8.0
Name: Food Environment Index, Length: 3093, dtype: float64

In [6]:
bottomten = clean_data2["Food Environment Index"].quantile(q=0.10)
bottomten

5.5

In [7]:
bins = [0, 5.5, 10]
ranges = ["Food Deserts", "Healthy"]

In [8]:
binned_if_desert = pd.cut(clean_data2["Food Environment Index"], bins, labels=ranges)


In [16]:
clean_data2["Environment Category"] = binned_if_desert
clean_data2.reset_index(drop=True,inplace=True)

clean_data2.head()

healthy_df = clean_data2.loc[clean_data2["Environment Category"] == "Healthy"]
healthy_df

desert_df = clean_data2.loc[clean_data2["Environment Category"] == "Food Deserts"]
desert_df

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude,Original Medicare,Environment Category,Inverse Index
2,Alabama,Barbour,27457,9820,24.533333,43123.000000,3,3,1,2,...,25,525.0,85.0,771.0,45.0,31.869603,-85.393197,4372,Food Deserts,5.1
5,Alabama,Bullock,10914,3745,19.333333,45273.666667,2,2,0,2,...,28,212.0,32.0,371.0,51.0,32.100554,-85.715697,1259,Food Deserts,6.2
11,Alabama,Choctaw,13859,5866,23.000000,48360.250000,1,1,0,2,...,25,425.0,14.0,399.0,34.0,32.019611,-88.263201,3124,Food Deserts,4.6
12,Alabama,Clarke,25833,10337,26.955556,44922.666667,3,3,1,4,...,23,508.0,110.0,940.0,31.0,31.676660,-87.830810,3915,Food Deserts,5.4
17,Alabama,Conecuh,13228,5625,29.400000,34702.800000,4,4,0,3,...,28,346.0,72.0,429.0,41.0,31.429272,-86.993674,2701,Food Deserts,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,Washington,Ferry,7551,3190,24.233333,44849.666667,2,2,1,1,...,19,141.0,12.0,251.0,31.0,48.470145,-118.516586,2148,Food Deserts,5.4
2915,Washington,Garfield,2266,989,12.400000,58750.000000,0,0,0,0,...,14,49.0,4.0,57.0,22.0,46.431559,-117.545169,625,Food Deserts,5.6
2938,Washington,Wahkiakum,3978,1737,21.900000,49523.000000,1,1,0,0,...,14,96.0,2.0,115.0,25.0,46.291134,-123.433470,1026,Food Deserts,4.7
2994,West Virginia,Wetzel,16583,6968,18.920000,50719.800000,3,3,1,2,...,24,277.0,13.0,721.0,29.0,39.605263,-80.639098,2476,Food Deserts,4.5


In [10]:


clean_data2 

#create inverse values so that the heat layer map shows green for access to food and red for no access.  

clean_data2["Inverse Index"] = clean_data2["Food Environment Index"].apply(lambda x: abs(x - 10))

clean_data2["Inverse Index"]

0       3.1
1       2.5
2       5.1
3       2.7
4       1.7
       ... 
3088    1.9
3089    2.0
3090    2.5
3091    2.0
3092    2.0
Name: Inverse Index, Length: 3093, dtype: float64

In [11]:
#print column headers to easy viewing when analyzing data
for col in clean_data2.columns:
    print(col)

State
County
POP2010
OHU2010
PovertyRate
MedianFamilyIncome
LILATracts_1And10
LILATracts_halfAnd10
LILATracts_1And20
LILATracts_Vehicle
LAPOP1_10
LAPOP05_10
LAPOP1_20
LALOWI1_10
LALOWI05_10
LALOWI1_20
lapophalf
TractKids
TractSeniors
TractWhite
TractBlack
TractAsian
TractNHOPI
TractAIAN
TractOMultir
TractHispanic
TractHUNV
TractSNAP
Food Environment Index
# Deaths
Years of Potential Life Lost Rate
% Obese
% Fair/Poor
# Diabetics
# Violent Crimes
# Unemployed
% Children in Poverty
Latitude
Longitude
Original Medicare
Environment Category
Inverse Index


Create heat map of US with data by county showing the Food Environment Index Inverse 0 is best, 10 is worst 

In [12]:

locations = clean_data2[["Latitude", "Longitude"]]

food_env = clean_data2["Inverse Index"]

In [13]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=food_env, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

# Obesity

create a heat map to show obesity % for US by county. 100% is red, 0% is faint green

In [ ]:
locations = data_loc[["Latitude", "Longitude"]]

obese = data_loc["% Obese"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=obese, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


create heat maps by State Montana. create dataframe of Just Montana

In [17]:
montana_healthy = healthy_df.loc[clean_data2["State"] == "Montana",:]

#montana_healthy.head()

montana_desert = desert_df.loc[clean_data2["State"] == "Montana",:]
montana_desert.head()

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude,Original Medicare,Environment Category,Inverse Index
1586,Montana,Big Horn,12865,4004,31.22,46813.2,4,4,1,1,...,22,193.0,28.0,372.0,40.0,45.423382,-107.489853,1607,Food Deserts,5.5
1587,Montana,Blaine,6491,2357,29.80,38925.5,1,1,1,1,...,20,100.0,10.0,120.0,40.0,48.432676,-108.958602,1004,Food Deserts,4.7
1590,Montana,Carter,1160,532,10.90,61429.0,0,0,0,0,...,11,13.0,0.0,23.0,23.0,45.516757,-104.536159,286,Food Deserts,5.6
1592,Montana,Chouteau,5813,2294,18.50,54260.0,1,1,1,1,...,17,74.0,6.0,75.0,23.0,47.880584,-110.435224,959,Food Deserts,5.7
1594,Montana,Daniels,1751,798,6.70,63603.0,0,0,0,0,...,10,27.0,2.0,25.0,10.0,48.783797,-105.548617,431,Food Deserts,4.8


In [22]:
#heat map of inverse data of Food Environment for Montana HEALTHY

locations2 = montana_healthy[["Latitude", "Longitude"]]


food_env2 = montana_healthy["Inverse Index"]

In [23]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=food_env2, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
#heat map of inverse data of Food Environment for Montana Desert

locations2 = montana_desert[["Latitude", "Longitude"]]


food_env3 = montana_desert["Inverse Index"]

In [21]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=food_env3, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#create heat map for Obese % for Montana Healthy

locations = montana_healthy[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
obese2 = montana_healthy["% Obese"]



In [ ]:

# Using the template add the Obese % marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt>Obesity %: </dt><dd>{% Obese}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
obese_info = [info_box_template1.format(**row) for index, row in montana_healthy.iterrows()]
locations = montana_healthy[["Latitude", "Longitude"]]

In [ ]:

# Plot Heatmap
fig = gmaps.figure()
marker_locations = montana_healthy[["Latitude", "Longitude"]]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=obese2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=.25)


# Add layer

fig.add_layer(heat_layer)
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig.add_layer(marker_layer2)
#Display figure
fig

In [ ]:
# Plot Heatmap
fig = gmaps.figure()
marker_locations = montana_healthy[["Latitude", "Longitude"]]

# Create heat layer
heat_layer2 = gmaps.heatmap_layer(locations2, weights=obese2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=.25)


# Add layer
marker_layer2 = gmaps.marker_layer(locations, info_box_content=obese_info)

fig.add_layer(heat_layer2)
fig.add_layer(markers)
fig.add_layer(marker_layer2)
#Display figure
fig

In [ ]:

gmaps.configure(api_key= g_key)

marker_locations = montana[["Latitude", "Longitude"]]
    

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
#Display figure
fig



In [ ]:

# Using the template add the food environment marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt>Food Environment Index: </dt><dd>{Food Environment Index}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
county_info = [info_box_template.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]

In [ ]:

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=county_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

heat_layer = gmaps.heatmap_layer(locations2, weights=food_env2, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)

fig.add_layer(heat_layer)
fig

# Display figure

In [ ]:

gmaps.configure(api_key= g_key)

marker_locations = montana[["Latitude", "Longitude"]]
    

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)

fig

In [ ]:

# Using the template add the food environment Index marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt>Food Environment Index: </dt><dd>{Food Environment Index}</dd>
</dl>
"""


# NOTE: be sure to update with your DataFrame name
county_info = [info_box_template.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]

# Diabetics

In [ ]:
locations = data_loc[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
diabetic = data_loc["# Diabetics"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=diabetic, 
                                 dissipating=False, max_intensity=3000,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


In [ ]:
locations = montana[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
diabetic2 = montana["# Diabetics"]

In [ ]:

# Using the template add the diabetic # marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt># of Diabetics: </dt><dd>{# Diabetics}</dd>
</dl>
"""


# NOTE: be sure to update with your DataFrame name
diabetic_info = [info_box_template.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]


In [ ]:
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=diabetic2, 
                                 dissipating=False, max_intensity=800,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)
marker_layer = gmaps.marker_layer(locations, info_box_content=diabetic_info)
fig.add_layer(marker_layer)
#Display figure
fig

# Unemployed

In [ ]:
locations = data_loc[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
unemployed = data_loc["# Unemployed"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=unemployed, 
                                 dissipating=False, max_intensity=3000,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig

In [ ]:
locations = montana[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
unemployed2 = montana["# Unemployed"]

In [ ]:
# Using the template add the # unemployed marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt># of Unemployed: </dt><dd>{# Unemployed}</dd>
</dl>
"""

# NOTE: be sure to update with your DataFrame name
employmemt_info = [info_box_template.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]

In [ ]:
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=unemployed2, 
                                 dissipating=False, max_intensity=800,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)
marker_layer = gmaps.marker_layer(locations, info_box_content=employmemt_info)
fig.add_layer(marker_layer)
#Display figure
fig